## Convert Entity Relationship Patterns to Fake Legal Sentences

In [2]:
import pandas as pd
import re
from collections import defaultdict
from pathlib import Path
import pathlib
import time
import random
from collections import defaultdict
import re
from flair.models import SequenceTagger
from flair.datasets import ColumnCorpus
from flair.data import Corpus
from flair.trainers import ModelTrainer
from flair.data import Sentence


In [3]:
# initialising primes and cache them
def isPrime(x):
    count = 0
    for i in range(int(x/2)):
        if x % (i+1) == 0:
            count = count+1
    return count == 1

MIN_PRIME = 0
MAX_PRIME = 1000

CACHED_PRIMES = [i for i in range(MIN_PRIME, MAX_PRIME) if isPrime(i)]

In [4]:
# CHANGE_ME BLOCK
# Change Me_1
#INPUT_FILE = r'../PATTERNS/CASE_LAW_COUNSEL/cl_counsel_re_patterns_DO_NOT_DELETE.xlsx'
#INPUT_FILE = r'../PATTERNS/CASE_LAW_COUNSEL/cl_counsel_re_patterns_DO_NOT_DELETE_4_annotation_Afsar_v1.xlsx'
INPUT_FILE = r'cl_counsel_re_patterns_DO_NOT_DELETE_4_annotation_Afsar_v4.xlsx'

DICTIONARIES_PATH = r'./'

# Change Me_2
MODEL_PATTERN_SHEET = 'ER_PATTERNS'
NAMED_ENTITIES_SHEET = 'NamedEntities'
#ENTITY_RELATIONSHIPS_SHEET = 'EntityRelationships'

VAL_TEST_SKIP_STEP = 3 # take every 3rd sentence into validation and test data

# Change Me_3
CORPUS_DIR_PATH = r'./PATTERNS/CORPUS_GROUPS'
#CORPUS_DIR_PATH = r'../PATTERNS/CORPUS_EXAMPLE'

# Change Me_4
PATTERN_GROUP_LIST = ['G1', 'G2','G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9',
                    'G10', 'G11', 'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 
                    'G18', 'G19', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 
                      'G26', 'G27', 'G28']
#PATTERN_GROUP_LIST = ['G1']
#PATTERN_GROUP_LIST = ['G2']


# Change Me_5
#MINI_BATCH_SIZE = 64
#MINI_BATCH_SIZE = 16
MINI_BATCH_SIZE = 64
BATCHES_PER_FILE = 3 # MINI_BATCH_SIZE * BATCHES_PER_FILE sentences for each pattern
START_TOKEN_INDEX = 1
# Set RANDOM_STATE to a fix prime value for reproducibale results
RANDOM_STATE = random.choice([i for i in CACHED_PRIMES if MIN_PRIME < i < MAX_PRIME])
print(RANDOM_STATE)

929


In [5]:
# ER Functions
def get_entity_code_df_dict():
    """Maps each EntityCode to a df that stores dictionary data for this EntityCode."""
    dict_files = NER_DF.DictionaryFileName.unique()
    ec_df_dict = {}
    for dict_file in dict_files:
        file_path = f'{DICTIONARIES_PATH}{dict_file}'
        dict_df = pd.read_excel(file_path)
        print(f'{dict_df.shape} : {dict_file}')
        df = NER_DF[NER_DF.DictionaryFileName == dict_file]
        entity_code_list = list(df.EntityCode.unique())
        for code in entity_code_list:
            ec_df_dict [code] = dict_df
            
    return ec_df_dict

In [6]:
# ER Functions
def get_entity_tag(entity_type):
    if NER_DF.loc[entity_type].isSupported:
        return NER_DF.loc[entity_type].EntityTag
    else: return 'other'
    
def get_entity_code_id_type_tag(entity):
    text = entity[1:-1] # exclude angular brackets
    entity_name = text.strip()
    entity_id = -1
        
    if "TYPE=" in text: # <TITLE_1 TYPE=GOV_TITLE> 
        parts = text.split("TYPE=")
        type_parts = parts[1].split() # <TITLE_1 TYPE=GOV_TITLE IS_PLURAL=TRUE>
        entity_type = type_parts[0].strip() # GOV_TITLE
        entity_name = parts[0].strip()
            
        splits = entity_name.split('_')
        if splits[-1].isnumeric():   # TITLE_2
            entity_id = int(splits[-1])  # id = 2
            last = len(splits) - 1
            entity_code = '_'.join(splits[0:last]) # base code = TITLE
        else: 
            entity_code = entity_name # STATE_ABBREV
    else:
        splits = entity_name.split('_')
        if splits[-1].isnumeric():   # <ATTORNEY_2>
            entity_id = int(splits[-1])  # id = 2
            last = len(splits) - 1
            entity_code = '_'.join(splits[0:last]) # base code = ATTORNEY
        else: 
            entity_code = entity_name # STATE_ABBREV
            
        entity_type = entity_code
        
    entity_tag = get_entity_tag(entity_type)
        
    return (entity_code, entity_id, entity_type, entity_tag)

def extract_entity_dict(pattern):
    """
    pattern = "<TITLE_1 TYPE=GOV_TITLE> <ATTORNEY_1> , by <TITLE_2 TYPE=GOV_TITLE> <ATTORNEY_2> , for the State <ATTORNEY_1> ."
    entity_list = ['<TITLE_1 TYPE=GOV_TITLE>', '<ATTORNEY_1>', '<TITLE_2 TYPE=GOV_TITLE>', '<ATTORNEY_2>']
    return: {'<TITLE_2 TYPE=GOV_TITLE>': ('TITLE', 2, 'GOV_TITLE', 'employmentTitle'),
             '<ATTORNEY_2>': ('ATTORNEY', 2, 'ATTORNEY', 'attorney'),
             '<ATTORNEY_1>': ('ATTORNEY', 1, 'ATTORNEY', 'attorney'),
             '<TITLE_1 TYPE=GOV_TITLE>': ('TITLE', 1, 'GOV_TITLE', 'employmentTitle')}
    """
    pattern = pattern.strip() # remove any extra spaces at head and tail
    # extract everything between angular braces
    entity_list = list(set(re.findall(r'\<[^>]*\>', pattern)))
    entity_dict = {}
    
    for entity in entity_list:
        entity_code, entity_id, entity_type, entity_tag = get_entity_code_id_type_tag(entity)
        entity_dict[entity] = (entity_code, entity_id, entity_type, entity_tag)
        #print(f'extract_entity_list: {entity} : code={entity_code} : id={entity_id} : type={entity_type}')
    return entity_dict

# Test: extract_entity_dict()
#pattern = "<TITLE_1 TYPE=GOV_TITLE> <ATTORNEY_1> , by <TITLE_2 TYPE=GOV_TITLE> <ATTORNEY_2> , for the State <ATTORNEY_1> ."
#extract_entity_dict(pattern)

In [7]:
# ER Functions
def get_fake_name_list(entity_code, entity_type, replace = True, how_many=64, random_state=RANDOM_STATE, fixed = False):
    """Sample with replacement; otherwise, may not have enough names to sample.""" 
    if not fixed: random_state = random.choice([i for i in CACHED_PRIMES if MIN_PRIME < i < MAX_PRIME])
    ser = ENTITY_CODE_DF_DCT[entity_code][entity_type]
    ser.dropna(inplace = True) # critical, as the excel files are of uneven row numbers per column
    name_list = list(ser.sample(n=how_many, replace=True, random_state=random_state).values)
    return name_list    

def generate_fake_entities_df(entity_dict, how_many=8):
    fake_dict = {}
    for entity, val in entity_dict.items():
        entity_code = val[0]
        entity_type = val[2] # entity_type
        entity_name_list = get_fake_name_list(entity_code, entity_type, replace = True, how_many=how_many, 
                                            random_state=RANDOM_STATE, fixed = False)
        #print(f'generate_fake_entities_df:{entity} : fake names = {len(entity_name_list)} : {entity_name_list}')
        fake_dict[entity] = entity_name_list

    return pd.DataFrame(fake_dict)

# Test: extract_entity_dict()
#pattern = "<TITLE_1 TYPE=GOV_TITLE> <ATTORNEY_1> , by <TITLE_2 TYPE=GOV_TITLE> <ATTORNEY_2> , for the State <ATTORNEY_1> ."
#entity_dict = extract_entity_dict(pattern)
#fake_entity_df = generate_fake_entities_df(entity_dict, how_many=3)
#fake_entity_df

In [8]:
# ER Functions
def get_pos_list(pattern):
    """
    pattern = "<TITLE_1 TYPE=GOV_TITLE> <ATTORNEY_1> , by <TITLE_2 TYPE=GOV_TITLE> <ATTORNEY_2> , for the State <ATTORNEY_1> ."
    return = left:  [0, 25, 43, 68, 97] and 
             right: [23, 36, 66, 79, 108]
    """
    left = '<'
    right = '>'
    left_pos_list = [m.start() for m in re.finditer(left, pattern)]
    right_pos_list = [m.start() for m in re.finditer(right, pattern)]
    
    #print(left_pos_list)
    #print(right_pos_list)
    return left_pos_list, right_pos_list
    
def tokenize_other_text(left_text, token_start):
    token_list = str(left_text).split()
    fake_list = []
    other_tag = 'O'
    for token in token_list:
        fake_list.append((token_start, token, other_tag))
        token_start = token_start + 1
        
    return fake_list, token_start

def tokenize_entity_text(entity_name, token_start, entity_tag):
    if entity_tag == 'other':
        fake_list, token_start = tokenize_other_text(str(entity_name), token_start)
        return fake_list, token_start
    
    token_list = str(entity_name).split()
    fake_list = []
    B_tag = 'B-' + entity_tag
    I_tag = 'I-' + entity_tag

    fake_list.append((token_start, token_list[0], B_tag))
    token_start = token_start + 1
    for idx in range(1, len(token_list)):
        token = token_list[idx]
        fake_list.append((token_start, token, I_tag))
        token_start = token_start + 1
        
    return fake_list, token_start

def get_entity_info(lpos, rpos, text, row, entity_dict):
    entity_text = text[lpos+1:rpos]  # entity text : TITLE_1 TYPE=GOV_TITLE>
    entity_col = "<" + entity_text + ">"
    entity_name = row[entity_col]
    entity_code = entity_dict[entity_col][0]  # the first tuple element is a code
    entity_id = entity_dict[entity_col][1]
    entity_tag = entity_dict[entity_col][-1]  # the last tuple element is a tag
    
    return entity_name, entity_code, entity_id, entity_tag

def get_relations_str(er_dict, pattern):
    """
    Example of relations str: 14;15;9;9;Located_In|24;26;14;15;OrgBased_In|24;26;41;41;OrgBased_In|24;26;9;9;OrgBased_In|28;32;9;9;OrgBased_In|28;32;14;15;OrgBased_In|28;32;41;41;OrgBased_In|35;35;9;9;OrgBased_In|35;35;41;41;OrgBased_In|35;35;14;15;OrgBased_In|41;41;9;9;Located_In
    """
    ROI_COLS = ['EntityCode_HEAD', 'HEAD_ID','EntityRelationship', 'EntityCode_TAIL', 'TAIL_ID']
    mask = MODEL_PATTERN_DF.PATTERN == pattern
    df = MODEL_PATTERN_DF [mask][ROI_COLS]
    if df.shape[0] == 0: return ""
    
    relations_str = "" 
    for idx, row in df.iterrows():
        er = row.EntityRelationship
        head_key = (row.EntityCode_HEAD, row.HEAD_ID)
        tail_key = (row.EntityCode_TAIL,row.TAIL_ID)
        for h_token_span_str in er_dict[head_key]:
            for t_token_span_str in er_dict[tail_key]:
                if len(relations_str) == 0: # the first time
                    relations_str = h_token_span_str + t_token_span_str + er
                else: relations_str = relations_str + "|" + h_token_span_str + t_token_span_str + er
                
    return relations_str

def create_fake_tagged_token_list (entity_dict, pattern, row, left_pos_list, right_pos_list):
    start = 0
    master_list = []
    token_start = START_TOKEN_INDEX
    fake_sent = ""
    er_dict = defaultdict(list)
    text = pattern
    
    # Iterate over non-entity and entity text segments
    # Tokenize each segment, and add the tag
    for idx, lpos in enumerate(left_pos_list):
        rpos = right_pos_list[idx]
        left_text = text[start:lpos] # text before entity
        # tokenize other, non-entity text: left or right to the entity text
        fake_other_list, token_start = tokenize_other_text(left_text, token_start)
        
        # tokenize entity text
        entity_name, entity_code, entity_id, entity_tag = get_entity_info(lpos, rpos, text, row, entity_dict)
        
        start_token = str(token_start) + ";"
        fake_entity_list, token_start = tokenize_entity_text(entity_name, token_start, entity_tag)
        end_token = str(token_start - 1) + ";"
        
        # token span string: "start;end" token id
        key = (entity_code, entity_id) # (TITLE, 2), (ATTORNEY, 3)
        token_span_str = str(start_token) + str(end_token)
        if key in er_dict.keys(): er_dict[key].append(token_span_str)
        else: er_dict[key] = [token_span_str]
        
        #print(entity_col, "\t: ", entity_tag, "\t: ", entity_code, entity_id, "\t: ", entity_name)
        
        master_list = [*master_list, *fake_other_list, *fake_entity_list]
        start = rpos + 1
        fake_sent = fake_sent + " " + left_text.strip() + " " + str(entity_name).strip()
    
    # process the remaining, rightmost, non-entity text
    right_text = text[start:len(pattern)]
    
    fake_other_list, token_start = tokenize_other_text(right_text, token_start)
    
    # create final master_list of (id, token, tag) triplets of the fake sentence
    master_list = [*master_list, *fake_other_list]
    
    # create conll sentence
    conll_sent = ""
    for token in master_list:
        conll_sent = conll_sent + f'{token[0]}\t{token[1]}\t{token[2]}\n'
    
    # create fake text sentence
    fake_sent = fake_sent + " " + right_text.strip()
    
    # create relationship string
    relations_str = get_relations_str(er_dict, pattern)
    
    return master_list, fake_sent.strip(), relations_str, conll_sent
    
def get_fake_tagged_tokens_df(entity_dict, fake_entity_df, pattern):
    left_pos_list, right_pos_list = get_pos_list(pattern)
    
    master_list = []
    for idx, row in fake_entity_df.iterrows():
        fake_tokens_list, fake_sent, relations_str, conll_sent = create_fake_tagged_token_list (entity_dict, 
                                                                pattern, row, left_pos_list, right_pos_list)
        master_list.append((pattern, fake_tokens_list, fake_sent, relations_str, conll_sent))
        
    fake_tokenized_sent_df = pd.DataFrame(master_list)
    fake_tokenized_sent_df.columns = ['PATTERN', 'FakeTaggedTokens', 'FakeSentence_TEXT', 'Relations_TEXT', 'FakeSentence_CONLL']
    return fake_tokenized_sent_df

#================================
# To test: get_fake_tokenized_sent_df()
#pattern = "<TITLE_1 TYPE=GOV_TITLE> <ATTORNEY_1> , by <TITLE_2 TYPE=GOV_TITLE> <ATTORNEY_2> , for the State <ATTORNEY_1> ."

#unique_pattern_list = list(MODEL_PATTERN_DF.PATTERN.unique())
#print(len(unique_pattern_list))

#idx = 1
#pattern = unique_pattern_list[idx]
#print(pattern)
#entity_dict = extract_entity_dict(pattern)
#print(entity_dict)
#fake_entity_df = generate_fake_entities_df(entity_dict, how_many=3)
#fake_tokenized_sent_df = get_fake_tagged_tokens_df(entity_dict, fake_entity_df, pattern)

# Test get_pos_list()
#pattern = "<TITLE_1 TYPE=GOV_TITLE> <ATTORNEY_1> , by <TITLE_2 TYPE=GOV_TITLE> <ATTORNEY_2> , for the State <ATTORNEY_1> ."
#left_pos_list, right_pos_list = get_pos_list(pattern)
#print(left_pos_list)
#print(right_pos_list)

#fake_tokenized_sent_df.head()
#fake_entity_df.head()

In [9]:
# ER Functions Testing
# unique_pattern_list = list(MODEL_PATTERN_DF.PATTERN.unique())
#print(len(unique_pattern_list))

#idx = 1
#pattern = unique_pattern_list[idx]

#for idx, pattern  in enumerate(unique_pattern_list):
#    entity_dict = extract_entity_dict(pattern)
    #print(entity_dict)
#    fake_entity_df = generate_fake_entities_df(entity_dict, how_many=3)
#    fake_tokenized_sent_df = get_fake_tagged_tokens_df(entity_dict, fake_entity_df, pattern)

#    row = fake_tokenized_sent_df.iloc[0]
#    print(f'{idx} pattern = {row.PATTERN}')
#    print('-'*50)
#    print(f'{idx} text = {row.FakeSentence_TEXT}')
#    print('-'*50)
#    print(f'{idx} relations = {row.Relations_TEXT}')
#    print('-'*50)
#    print(row.FakeSentence_CONLL)
#    print()
#    print("="*50)


In [10]:
# ER Functions
def process_er_patterns(pg_list = PATTERN_GROUP_LIST, 
                        merge = False, pattern_src = "cl_counsel", 
                        conll=True):
    """For a given pattern groupID from the pg_list, generate fake sentences and save them in conll format.
    merge = True : combine output into a single file for different groupIDs
    merge = False: do not combine into a single conll file
    """
    
    pattern_sent_dict = defaultdict(list)
    print(len(pg_list), " : The Number of Pattern Groups to Process.")
    for pattern_group in pg_list:
        print("="*50)
        print(pattern_group, ": Processing this pattern group")
        df = MODEL_PATTERN_DF [MODEL_PATTERN_DF.GroupID == pattern_group]
        pattern_list = df.PATTERN.unique()
        print(pattern_group, ": The Number of Unique Patterns : ", len(pattern_list))
                      
        file_prefix = pattern_group
        if merge: file_prefix = '_'.join(pg_list)
    
        file_names = [f'{file_prefix}_{pattern_src}_train_iob.txt', 
                      f'{file_prefix}_{pattern_src}_val_iob.txt',
                      f'{file_prefix}_{pattern_src}_test_iob.txt']
        
        # Validation and Test data set do not have to be large for each pattern.
        # Taking only every third sentence might suffice.
        file_skip_dict = {file_names[0]:1, file_names[1]:VAL_TEST_SKIP_STEP, file_names[2]:VAL_TEST_SKIP_STEP} # how many sentences to skip
                      
        no_files = len(file_names)
        how_many = MINI_BATCH_SIZE * no_files * BATCHES_PER_FILE
        print('HOW MANY = ', how_many) # 384 = 64 * 3 * 2

        for pid, pattern in enumerate(pattern_list):
            print('='*30)
            print(pattern_group, ": ", pid, ": PATTERN TEMPLATE: \n\t", pattern)
            print('-'*30)
            
            entity_dict = extract_entity_dict(pattern)
            print(pattern_group, ": ", pid, ": ENTITY (CODE, ID, TYPE): \n\t", entity_dict)
            fake_entity_df = generate_fake_entities_df(entity_dict, how_many=how_many)
            fake_tokenized_sent_df = get_fake_tagged_tokens_df(entity_dict, fake_entity_df, pattern)
            
            if conll:
                no_sentences = fake_tokenized_sent_df.shape[0]
                print(no_sentences, ' fake sentences per file and per pattern')
                start = 0
                sent_per_file = no_sentences // no_files
                print(pid, "\tPreparing CONLLU file format for writing...")
                for file_name in file_names:
                    print(pid, "\t",file_name)
                    end = start + sent_per_file
                    skip = file_skip_dict[file_name]
                    conllu_text = convert_fake_sent_to_CONLLU_sentence(fake_tokenized_sent_df.iloc[start:end:skip]) # CHANGE ME
                    write_to_conllu_file(conllu_text,  file_name,  dir_path=CORPUS_DIR_PATH, mode="a", verbose = False)
                    start = end
                    
    return pattern_sent_dict

def convert_fake_sent_to_CONLLU_sentence(fake_tokenized_sent_df):
    text = ""
    
    for _, row in fake_tokenized_sent_df.iterrows():
        pattern = f'# pattern = {row.PATTERN}\n'
        sentence = f'# text = {row.FakeSentence_TEXT}\n'
        relations = f'# relations = {row.Relations_TEXT}\n'
        conll = row.FakeSentence_CONLL + '\n'
        
        text = text + pattern + sentence + relations + conll

    return text

def write_to_conllu_file(text, file_name="conllu_iob.txt", dir_path=CORPUS_DIR_PATH, mode = 'a', verbose = False):
    path = pathlib.Path(dir_path)
    abs_path = path.absolute()
    if not abs_path.exists():
        abs_path.mkdir()
    file_path = abs_path.joinpath(file_name)
        
    with open(file_path, mode, encoding = "utf-8") as the_file:
        the_file.write(text)
        
    if verbose: print("Saved: ", file_path)
    return

### Step-1: Read and Filter Model Pattern File

__(a) Load Named Entities Table__

In [11]:
NER_DF = pd.read_excel(INPUT_FILE, sheet_name = NAMED_ENTITIES_SHEET)
NER_DF.tail(7)

,Category,EntityCode,EntityType,EntityTag,isSupported,Example,DictionaryFileName,TO DO
32,Person,PERSON,PERSON,person,False,NaN,person_names.xlsx,NaN
33,Person,PERSON_INITIALS,PERSON_INITIALS,person,False,A . B .,person_names.xlsx,NaN
34,Person,PERSON_LAST_NAME,PERSON_LAST_NAME,person,False,NaN,person_names.xlsx,NaN
35,Employment,TITLE,ATTY_TITLE,employmentTitle,True,NaN,US_employment_titles.xlsx,NaN
36,Employment,TITLE,GOV_TITLE,employmentTitle,True,NaN,US_employment_titles.xlsx,NaN
37,Organization,PRISON,PRISON,prison,False,NaN,US_organization.xlsx,NaN
38,Organization,CHURCH,CHURCH,church,False,NaN,US_organization.xlsx,NaN


In [12]:
NER_DF.head(12)

,Category,EntityCode,EntityType,EntityTag,isSupported,Example,DictionaryFileName,TO DO
0,Litigant,LITIGANT,LITIGANT_PERSON,litigant,False,NaN,US_litigants.xlsx,NaN
1,Litigant,LITIGANT,LITIGANT_COMPANY,litigant,False,NaN,US_litigants.xlsx,NaN
2,Litigant,LITIGANT,LITIGANT_COUNTY,litigant,False,NaN,US_litigants.xlsx,NaN
3,Litigant,LITIGANT,LITIGANT_CHURCH,litigant,False,NaN,US_litigants.xlsx,NaN
4,Litigant,LITIGANT,LITIGANT_HOSPITAL,litigant,False,NaN,US_litigants.xlsx,NaN
5,Litigant,LITIGANT,LITIGANT_INSURANCE_CO,litigant,False,NaN,US_litigants.xlsx,NaN
6,Litigant,LITIGANT,LITIGANT_FOUNDATION,litigant,False,NaN,US_litigants.xlsx,NaN
7,Litigant,LITIGANT,GOV_LITIGANT,litigant,False,NaN,US_litigants.xlsx,NaN
8,Litigant,LITIGANT,LITIGANT_LAW_FIRM,litigant,False,NaN,US_litigants.xlsx,NaN
9,Litigant,LITIGANT_ROLE,LITIGANT_ROLE,litigantRole,False,NaN,US_litigants.xlsx,NaN


In [13]:
#NER_DF.set_index('EntityCode', inplace = True) # previously for named entities
NER_DF.set_index('EntityType', inplace = True)
NER_DICT = NER_DF.to_dict(orient='index')
print(NER_DICT['ATTORNEY'])
NER_DF.head()

{'Category': 'Attorney', 'EntityCode': 'ATTORNEY', 'EntityTag': 'attorney', 'isSupported': True, 'Example': nan, 'DictionaryFileName': 'US_attorney_names.xlsx', 'TO DO': nan}


,Category,EntityCode,EntityTag,isSupported,Example,DictionaryFileName,TO DO
EntityType,,,,,,,
LITIGANT_PERSON,Litigant,LITIGANT,litigant,False,NaN,US_litigants.xlsx,NaN
LITIGANT_COMPANY,Litigant,LITIGANT,litigant,False,NaN,US_litigants.xlsx,NaN
LITIGANT_COUNTY,Litigant,LITIGANT,litigant,False,NaN,US_litigants.xlsx,NaN
LITIGANT_CHURCH,Litigant,LITIGANT,litigant,False,NaN,US_litigants.xlsx,NaN
LITIGANT_HOSPITAL,Litigant,LITIGANT,litigant,False,NaN,US_litigants.xlsx,NaN


In [14]:
NER_DF.index

Index(['LITIGANT_PERSON', 'LITIGANT_COMPANY', 'LITIGANT_COUNTY',
       'LITIGANT_CHURCH', 'LITIGANT_HOSPITAL', 'LITIGANT_INSURANCE_CO',
       'LITIGANT_FOUNDATION', 'GOV_LITIGANT', 'LITIGANT_LAW_FIRM',
       'LITIGANT_ROLE', 'PRISON_ID', 'ATTORNEY', 'BAR_ID', 'LAW_FIRM',
       'ODD_LAW_FIRM', 'UNIQUE_LAW_FIRM', 'GOVT_ORG_AS_LAW_FIRM',
       'AMICUS_CURAE_LAW_FIRM', 'FOUNDATION_LAW_FIRM', 'BRANCH_CITY_ABBR',
       'BRANCH_STATE_NAME', 'BRANCH_STATE_ABBR', 'BRANCH_STATE',
       'BRANCH_LANDMARK', 'BRANCH_CITY', 'CITY', 'CITY_ABBR', 'STATE',
       'STATE_NAME', 'STATE_ABBR', 'LANDMARK', 'COUNTY', 'PERSON',
       'PERSON_INITIALS', 'PERSON_LAST_NAME', 'ATTY_TITLE', 'GOV_TITLE',
       'PRISON', 'CHURCH'],
      dtype='object', name='EntityType')

__(b) Load Corresponding Dictionaries__

In [15]:
ENTITY_CODE_DF_DCT = get_entity_code_df_dict()
ENTITY_CODE_DF_DCT.keys()

(61002, 11) : US_litigants.xlsx
(61002, 3) : US_attorney_names.xlsx
(221, 7) : US_law_firms.xlsx
(28889, 6) : US_branch_loc.xlsx
(28889, 2) : US_cities_state_names.xlsx
(55, 5) : US_cities_abbrev_names.xlsx
(55, 3) : US_state_abbrev_names.xlsx
(3241, 2) : US_county_state_names.xlsx
(61002, 8) : person_names.xlsx
(50, 3) : US_employment_titles.xlsx
(6, 2) : US_organization.xlsx


dict_keys(['LITIGANT', 'LITIGANT_ROLE', 'PRISON_ID', 'ATTORNEY', 'BAR_ID', 'LAW_FIRM', 'BRANCH_LOC', 'CITY', 'CITY_ABBR', 'LANDMARK', 'STATE', 'STATE_NAME', 'STATE_ABBR', 'COUNTY', 'PERSON', 'PERSON_INITIALS', 'PERSON_LAST_NAME', 'TITLE', 'PRISON', 'CHURCH'])

In [16]:
# Values are numeric not text
ENTITY_CODE_DF_DCT['PRISON_ID'] = ENTITY_CODE_DF_DCT['PRISON_ID'].astype({"PRISON_ID": int}, errors='ignore')
#ENTITY_CODE_DF_DCT['PRISON_ID'].PRISON_ID[0:5]

In [17]:
ENTITY_CODE_DF_DCT['LITIGANT_ROLE'].LITIGANT_ROLE[5:10]

5                appellants
6    APPELLANT / PETITIONER
7    Appellant / Petitioner
8    appellant / petitioner
9    APPELLANT - PETITIONER
Name: LITIGANT_ROLE, dtype: object

In [18]:
#ENTITY_CODE_DF_DCT['LAW_FIRM'].head(3)

__(c) Load Entity Relationship Patterns__

In [19]:
MODEL_PATTERN_DF = pd.read_excel(INPUT_FILE, sheet_name = MODEL_PATTERN_SHEET)

# filter er patterns that are completed
mask1 = MODEL_PATTERN_DF.Status == "DONE"
mask2 = MODEL_PATTERN_DF.Status == "SME"
mask = mask1 | mask2
MODEL_PATTERN_DF = MODEL_PATTERN_DF[mask] 
MODEL_PATTERN_DF.reset_index(inplace = True)

# Which groups are supported
print(f'GroupID: {MODEL_PATTERN_DF.GroupID.unique()}')

# change column types to integer
MODEL_PATTERN_DF['HEAD_ID'] = MODEL_PATTERN_DF['HEAD_ID'].astype({"HEAD_ID": int}, errors='ignore')
MODEL_PATTERN_DF['TAIL_ID'] = MODEL_PATTERN_DF['TAIL_ID'].astype({"TAIL_ID": int}, errors='ignore')
MODEL_PATTERN_DF['PatternID'] = MODEL_PATTERN_DF['PatternID'].astype({"PatternID": int}, errors='ignore')

GroupID: ['G18' 'G13' 'G6' 'G27' 'G9' 'G14' 'G28' 'G1' 'G20' 'G15' 'G11' 'G12'
 'G10' 'G17' 'G2' 'G4' 'G5' 'G25' 'G22' 'G23' 'G24' 'G8']


In [20]:
MODEL_PATTERN_DF.head(3)

,index,Status,PatternID,EntityCode_HEAD,HEAD_ID,EntityRelationship,EntityCode_TAIL,TAIL_ID,PATTERN,ExceptionEntityName,ExceptionEntityTag,GroupID,Sentence_TEXT,isSpecial,Sentence_SRC,Sentence_LIST,Sentence_SRC_LIST
0,0,DONE,0,ATTORNEY,2.0,atty_title,TITLE,2.0,"<TITLE_1 TYPE=GOV_TITLE> <ATTORNEY_1> , by <T...",NaN,NaN,G18,"Attorney General Joshua H . Stein , by Special...",False,"September 21, 2021 Batch",[],[]
1,1,DONE,0,ATTORNEY,1.0,atty_title,TITLE,1.0,"<TITLE_1 TYPE=GOV_TITLE> <ATTORNEY_1> , by <T...",NaN,NaN,G18,"Attorney General Joshua H . Stein , by Special...",False,"September 21, 2021 Batch",[],[]
2,2,DONE,1,ATTORNEY,2.0,atty_title,TITLE,2.0,"<ATTORNEY_1> , <TITLE_1 TYPE=GOV_TITLE> , and...",NaN,NaN,G13,"Remington S . Dalke , county attorney , and De...",False,"September 21, 2021 Batch",[],[]


In [21]:
# These numbers should be the same
len(MODEL_PATTERN_DF.PatternID.unique()), len(MODEL_PATTERN_DF.PATTERN.unique())

(176, 176)

In [22]:
len(MODEL_PATTERN_DF.PatternID.unique())

176

In [23]:
MODEL_PATTERN_DF.columns

Index(['index', 'Status', 'PatternID', 'EntityCode_HEAD', 'HEAD_ID',
       'EntityRelationship', 'EntityCode_TAIL', 'TAIL_ID', 'PATTERN',
       'ExceptionEntityName', 'ExceptionEntityTag', 'GroupID', 'Sentence_TEXT',
       'isSpecial', 'Sentence_SRC', 'Sentence_LIST', 'Sentence_SRC_LIST'],
      dtype='object')

In [24]:
MODEL_PATTERN_DF

,index,Status,PatternID,EntityCode_HEAD,HEAD_ID,EntityRelationship,EntityCode_TAIL,TAIL_ID,PATTERN,ExceptionEntityName,ExceptionEntityTag,GroupID,Sentence_TEXT,isSpecial,Sentence_SRC,Sentence_LIST,Sentence_SRC_LIST
0,0,DONE,0,ATTORNEY,2.0,atty_title,TITLE,2.0,"<TITLE_1 TYPE=GOV_TITLE> <ATTORNEY_1> , by <T...",NaN,NaN,G18,"Attorney General Joshua H . Stein , by Special...",False,"September 21, 2021 Batch",[],[]
1,1,DONE,0,ATTORNEY,1.0,atty_title,TITLE,1.0,"<TITLE_1 TYPE=GOV_TITLE> <ATTORNEY_1> , by <T...",NaN,NaN,G18,"Attorney General Joshua H . Stein , by Special...",False,"September 21, 2021 Batch",[],[]
2,2,DONE,1,ATTORNEY,2.0,atty_title,TITLE,2.0,"<ATTORNEY_1> , <TITLE_1 TYPE=GOV_TITLE> , and...",NaN,NaN,G13,"Remington S . Dalke , county attorney , and De...",False,"September 21, 2021 Batch",[],[]
3,3,DONE,1,ATTORNEY,1.0,atty_title,TITLE,1.0,"<ATTORNEY_1> , <TITLE_1 TYPE=GOV_TITLE> , and...",NaN,NaN,G13,"Remington S . Dalke , county attorney , and De...",False,"September 21, 2021 Batch",[],[]
4,4,DONE,41,NaN,NaN,NaN,NaN,NaN,"<ATTORNEY_1> , under appointment by the Court...",NaN,NaN,G6,"Lori Siegel , under appointment by the Court o...",False,"September 21, 2021 Batch",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,713,DONE,199,NaN,NaN,NaN,NaN,NaN,"Ms . <ATTORNEY_1> , Attorney at Law , For <LIT...",NaN,NaN,G2,"Ms . Jessica Carter , Attorney at Law , For St...",False,"September 21, 2021 Batch",[],[]
596,714,DONE,200,NaN,NaN,NaN,NaN,NaN,"Ms . <ATTORNEY_1> , Attorney at Law , For The ...",NaN,NaN,G2,"Ms . Kimberly M . Marston , Attorney at Law , ...",False,"September 21, 2021 Batch",[],[]
597,715,DONE,202,NaN,NaN,NaN,NaN,NaN,"Ms .<ATTORNEY_1> , Attorney at Law , For Count...",NaN,NaN,G2,"Ms . Monica Langdon Jackson , Attorney at Law ...",False,"September 21, 2021 Batch",[],[]
598,716,DONE,203,ATTORNEY,1.0,atty_city,CITY,1.0,"Physicians for Informed Consent , Other : <ATT...",NaN,NaN,G2,"Physicians for Informed Consent , Other : Greg...",False,"September 21, 2021 Batch",[],[]


In [25]:
# Check which patterns are redundant
tmp_cols = ['EntityCode_HEAD', 'HEAD_ID',
       'EntityRelationship', 'EntityCode_TAIL', 'TAIL_ID', 'PATTERN']
tmp_df = MODEL_PATTERN_DF[tmp_cols].drop_duplicates()
s1 = set(tmp_df.index)
s2 = set(MODEL_PATTERN_DF.index)
diff = list(s2.difference(s1))
print(len(diff))
for row in diff:
    print(MODEL_PATTERN_DF.loc[row].PATTERN)

0


In [26]:
# Check which patterns are redundant
tmp_cols = ['EntityCode_HEAD', 'HEAD_ID',
       'EntityRelationship', 'EntityCode_TAIL', 'TAIL_ID', 'PATTERN']
tmp_df = MODEL_PATTERN_DF[tmp_cols].drop_duplicates()
s1 = set(tmp_df.index)
s2 = set(MODEL_PATTERN_DF.index)
diff = list(s1.difference(s2))
print(len(diff))
for row in diff:
    print(MODEL_PATTERN_DF.loc[row].PATTERN)

0


In [27]:
# Identify all the unique patterns
UNIQUE_PATTERN_LIST = MODEL_PATTERN_DF.PATTERN.unique()
print(len(UNIQUE_PATTERN_LIST))

176


In [28]:
# Find out why two different patterns have the same pattern ID
# This should print nothing
unique_pid_list = MODEL_PATTERN_DF.PatternID.unique()
for pid in unique_pid_list:
    mask = MODEL_PATTERN_DF.PatternID == pid
    tmp_df = MODEL_PATTERN_DF[mask]
    no_patterns = len(tmp_df.PATTERN.unique())
    if no_patterns > 1:
        print(pid)

In [29]:
PATTERN_GROUP_LIST = MODEL_PATTERN_DF.GroupID.unique()
#PATTERN_GROUP_LIST = ['G13']
PATTERN_GROUP_LIST

array(['G18', 'G13', 'G6', 'G27', 'G9', 'G14', 'G28', 'G1', 'G20', 'G15',
       'G11', 'G12', 'G10', 'G17', 'G2', 'G4', 'G5', 'G25', 'G22', 'G23',
       'G24', 'G8'], dtype=object)

In [30]:
#PATTERN_GROUP_LIST = ['G28']

In [31]:
# Create fake sentences and save to CONLL or CONLLU format for entity relationship (er) patterns
CONLLU = True
pattern_sent_dict = process_er_patterns(pg_list = PATTERN_GROUP_LIST, 
                                        merge = False, 
                                        pattern_src = "cl_counsel", conll=CONLLU)
print("COMPLETED NON-SPECIAL PATTERNS")

22  : The Number of Pattern Groups to Process.
G18 : Processing this pattern group
G18 : The Number of Unique Patterns :  4
HOW MANY =  576
G18 :  0 : PATTERN TEMPLATE: 
	  <TITLE_1 TYPE=GOV_TITLE> <ATTORNEY_1> , by <TITLE_2 TYPE=GOV_TITLE> <ATTORNEY_2> , for the State .   
------------------------------
G18 :  0 : ENTITY (CODE, ID, TYPE): 
	 {'<ATTORNEY_1>': ('ATTORNEY', 1, 'ATTORNEY', 'attorney'), '<TITLE_2 TYPE=GOV_TITLE>': ('TITLE', 2, 'GOV_TITLE', 'employmentTitle'), '<ATTORNEY_2>': ('ATTORNEY', 2, 'ATTORNEY', 'attorney'), '<TITLE_1 TYPE=GOV_TITLE>': ('TITLE', 1, 'GOV_TITLE', 'employmentTitle')}
576  fake sentences per file and per pattern
0 	Preparing CONLLU file format for writing...
0 	 G18_cl_counsel_train_iob.txt
0 	 G18_cl_counsel_val_iob.txt
0 	 G18_cl_counsel_test_iob.txt
G18 :  1 : PATTERN TEMPLATE: 
	 <ATTORNEY_1> , <TITLE_1 TYPE=GOV_TITLE> , <ATTORNEY_2> , <TITLE_2 TYPE=GOV_TITLE> , <ATTORNEY_3> , <TITLE_3 TYPE=GOV_TITLE> , <ATTORNEY_4> and <ATTORNEY_5> ,  <TITLE_4 TYPE

In [32]:

DOES_NOT_WORK = []

# Epoch - 1
# PATTERN_GROUP_LIST = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9',
#                       'G10', 'G11', 'G12', 'G13', 'G14', 'G15', 'G16', 'G17',
#                       'G18', 'G19', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26']

# Epoch - 2
# PATTERN_GROUP_LIST = ['G2', 'G3', 'G4', 'G6', 'G7', 'G8',
                    #   'G10', 'G11', 'G12', 'G14', 'G17']

# Epoch - 3
# PATTERN_GROUP_LIST = ['G2', 'G3', 'G4', 'G6', 'G7', 'G11', 'G14', 'G17']

# Epoch - 4 and 5
# PATTERN_GROUP_LIST = ['G7', 'G17', 'G2', 'G3']

# Epoch - 6
PATTERN_GROUP_LIST = ['G2', 'G3']

***
__STOP HERE__

In [33]:
DATA_FOLDER = 'PATTERNS/CORPUS_GROUPS'
DATA_PATH = Path(DATA_FOLDER)
MODEL_PATH = 'PATTERNS/MODELS_GROUPS'
COLUMNS = {0: 'text', 1: 'ner'}

print(PATTERN_GROUP_LIST)
MAX_EPOCHS = 1

for group in PATTERN_GROUP_LIST:
    print(group, ': TRAINING FOR THIS GROUP OF PATTERNS')
    TAGGER: SequenceTagger = SequenceTagger.load("final-model.pt")
    TAGGER.tag_dictionary.add_unk = True

    TRAIN_FILE = f'{group}_cl_counsel_train_iob.txt'
    TEST_FILE = f'{group}_cl_counsel_test_iob.txt'
    DEV_FILE = f'{group}_cl_counsel_val_iob.txt'
    print(f'{TRAIN_FILE} file is used for training')

    corpus: Corpus = ColumnCorpus(DATA_PATH, COLUMNS,
                                  train_file=TRAIN_FILE,
                                  test_file=TEST_FILE,
                                  dev_file=DEV_FILE,
                                  in_memory=False)
    # tag_dictionary = corpus.make_label_dictionary(label_type=TAG_TYPE)

    trainer: ModelTrainer = ModelTrainer(TAGGER, corpus)



    trainer.train(MODEL_PATH,
                  learning_rate=0.1,
                  mini_batch_size=64,
                  max_epochs=MAX_EPOCHS,
                  monitor_train=False,
                  monitor_test=False,
                  checkpoint=True,
                  train_with_dev=True,
                  embeddings_storage_mode='none')
    print(group,': COMPLETED MODEL TRAINING FOR THIS GROUP')
    # print(group, ': Loading the new model for the next group')
    # TAGGER: SequenceTagger = SequenceTagger.load(PRE_TRAINED_MODEL)
    # print(TAGGER.tag_dictionary)
    print("-"*30)


['G2', 'G3']
G2 : TRAINING FOR THIS GROUP OF PATTERNS
2021-12-03 23:32:06,831 loading file final-model.pt
G2_cl_counsel_train_iob.txt file is used for training
2021-12-03 23:32:06,967 Reading data from PATTERNS/CORPUS_GROUPS
2021-12-03 23:32:06,968 Train: PATTERNS/CORPUS_GROUPS/G2_cl_counsel_train_iob.txt
2021-12-03 23:32:06,968 Dev: PATTERNS/CORPUS_GROUPS/G2_cl_counsel_val_iob.txt
2021-12-03 23:32:06,968 Test: PATTERNS/CORPUS_GROUPS/G2_cl_counsel_test_iob.txt
2021-12-03 23:32:07,201 ----------------------------------------------------------------------------------------------------
2021-12-03 23:32:07,202 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): Lan

In [1]:
PRE_TRAINED_MODEL="final-model.pt"
TAGGER: SequenceTagger = SequenceTagger.load(PRE_TRAINED_MODEL)
TAGGER.tag_dictionary.add_unk = True # To handle unknown tags, such as S-attorney
from flair.datasets.relation_extraction import CoNLLUCorpus
corpus: Corpus = CoNLLUCorpus(
                    data_folder=DATA_PATH,
                    train_file=TRAIN_FILE,
                    dev_file=DEV_FILE,
                    test_file=TEST_FILE,
                    fields=['id', 'form', 'ner'],
)
current_tag_list = TAGGER.tag_dictionary.get_items()
corpus_tag_dictionary = corpus.make_label_dictionary(label_type = TAG_TYPE)
corpus_tag_dictionary.get_items()
sent = """For Andrew Saul , Commissioner of Social Security , Defendant : Jonathan Holland Hambrick , LEAD ATTORNEY , Office of the U . S . Attorney , Richmond , VA ."""
print(sent)
sentence = Sentence (sent)
TAGGER: SequenceTagger = SequenceTagger.load(PRE_TRAINED_MODEL)
TAGGER.predict(sentence)
sentence.get_spans("ner")


NameError: name 'SequenceTagger' is not defined